## What is this script good for ?
Consensus files of TE sequences often need a tedious pre-processing : LTR transposable elements are splitted between their intern and LTR parts. If you want to use those TE sequences for an alignment, you have to restore the real sequence of these TEs by flanking the intern part with the LTR part (like this : LTR|Intern|LTR ).

The goal of this script is to simplify this task and to provide a correctly reconstructed new fasta file.

## How to use it ?

This script need two files : a classic **consensus fasta file** (an example being the Dfam families.fa file present in the folder) and a **dictionnary in tsv format** providing correspondance between intern and LTR parts.

1) Generating the dictionnary

You can choose to write the dictionnary yourself (TE_name, TE_intern_part, TE_LTR_part, tab_separated):

dictionnary example :  
`Copia  Copia_I    Copia_LTR`  
`Roo   Roo-I_DM   Roo-LTR_DM`

OR generate it automatically using 'generate_dictionnary.py' script and the default list of suffixes ("standard_suffixes.tsv"):

`./generate_dictionnary.py -f families.fa --suffix standard_suffixes.tsv -o families.dictionnary.tsv`

You can also specify a custom list of suffixes in a tsv file, with the first line being the list of intern suffixes, and the second line being the list of LTR suffixes, separated with tabs.

ex :  
`_I -I_DM`  
`_LTR   -LTR_DM`

**It is advised to have a look at the generated dictionnary and manually cure it if needed.**

2) Get the new consensus fasta file using this command:

`./generate_new_consensus_fasta.py -f families.fa -d families.dictionnary -o new_families.fa`

Notes :

All TE sequences that are present in the fasta must be represented in the dictionnary.
TODO : Maybe add the option to directly output TE sequences that are not written in the dictionnary.
This can be easily implemented and tested by removing a random TE from the dictionnary...



In [138]:
# import warnings
# from warnings import warn
import logging
from Bio import SeqIO

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Dfam_consensus_fasta = "example_data/families.fa"
suffix_file = "standard_suffixes.tsv"



In [84]:
## Dictionnary generator

def get_TE_ID(record):
    return record.split()[-1]

def generate_matching_pairs_from_suffixes(consensus_fasta, suffix_file):
    with open(suffix_file, 'r') as input:
        I_suffix_list = input.readline().split()
        LTR_suffix_list = input.readline().split()
    I_suffix_set = set(I_suffix_list)
    LTR_suffix_set = set(LTR_suffix_list)

    def is_LTR_part(name):
        for suffix in LTR_suffix_set:
            if name.endswith(suffix):
                return True
        return False

    def is_I_part(name):
        for suffix in I_suffix_set:
            if name.endswith(suffix):
                return True
        return False

    def remove_suffix(seq_ID):
        suffix_list = list(LTR_suffix_set) + list(I_suffix_set)
        for suffix in suffix_list:
            if seq_ID.endswith(suffix):
                return seq_ID[:-len(suffix)]
        return seq_ID

    I_part_dict = {}
    LTR_part_dict = {}
    not_splitted_TE = []
    detected_conflicts = []

    records = list(SeqIO.parse(consensus_fasta, "fasta"))
    for record in records:
        TE_ID = get_TE_ID(record.description)
        TE_name = remove_suffix(TE_ID)
        if is_LTR_part(TE_ID):
            if TE_name in LTR_part_dict :
                detected_conflicts.append(TE_name)
                continue
            LTR_part_dict[TE_name] = TE_ID
        elif is_I_part(TE_ID):
            if TE_name in I_part_dict :
                detected_conflicts.append(TE_name)
                continue
            I_part_dict[TE_name] = TE_ID
        else:
            not_splitted_TE.append(TE_ID)

    if detected_conflicts :
        logging.warning("Detected conflict, following TEs can be matched with several suffixes : " + ', '.join(set(detected_conflicts)) + "\nYou should manually set it in the dictionnary.")
    
    matching_TE_parts = set(I_part_dict.keys()).intersection(set(LTR_part_dict.keys()))
    unmatched_I_list = [I_part_dict[x] for x in set(I_part_dict.keys()).difference(set(LTR_part_dict.keys()))]
    unmatched_LTR_list = [LTR_part_dict[x] for x in set(LTR_part_dict.keys()).difference(set(I_part_dict.keys()))]
    if len(unmatched_I_list + unmatched_LTR_list) > 0 :
        logging.warning("These TE names ends with a recognized suffix, but couldn't be matched. You might want to manually cure some of them in the dictionnary :\n" + ', '.join(unmatched_I_list) + "\n" + ', '.join(unmatched_LTR_list) + "\n")
    TE_dictionnary = ""
    for TE_name in matching_TE_parts:
        TE_dictionnary += "\t".join([TE_name, I_part_dict[TE_name], LTR_part_dict[TE_name]]) + "\n"
    TE_dictionnary += '\n'.join(unmatched_I_list + unmatched_LTR_list + not_splitted_TE)
    return TE_dictionnary

print(generate_matching_pairs_from_suffixes(Dfam_consensus_fasta, suffix_file))


ERROR:root:These TE names ends with a recognized suffix, but couldn't be matched. You might want to manually cure some of them in the dictionnary :
HMSBEAGLE_I, TOM_I
Stalker3_LTR, Gypsy6A_LTR, DMTOM1_LTR, DM412B_LTR, Gypsy12A_LTR

Invader6	Invader6_I	Invader6_LTR
BATUMI	BATUMI_I	BATUMI_LTR
MICROPIA	MICROPIA_I	MICROPIA_LTR
NOMAD	NOMAD_I	NOMAD_LTR
ZAM	ZAM_I	ZAM_LTR
Gypsy9	Gypsy9_I	Gypsy9_LTR
ROVER	ROVER-I_DM	ROVER-LTR_DM
STALKER4	STALKER4_I	STALKER4_LTR
MDG1	MDG1_I	MDG1_LTR
Copia2	Copia2_I	Copia2_LTR_DM
Gypsy5	Gypsy5_I	Gypsy5_LTR
MDG3	MDG3_I	MDG3_LTR
DIVER	DIVER_I	DIVER_LTR
GTWIN	GTWIN_I	GTWIN_LTR
BURDOCK	BURDOCK_I	BURDOCK_LTR
Gypsy7	Gypsy7_I	Gypsy7_LTR
IDEFIX	IDEFIX_I	IDEFIX_LTR
Invader1	Invader1_I	Invader1_LTR
Chouto	Chouto_I	Chouto_LTR
BLASTOPIA	BLASTOPIA_I	BLASTOPIA_LTR
Bica	Bica_I	Bica_LTR
Invader2	Invader2_I	Invader2_LTR
QUASIMODO2	QUASIMODO2-I_DM	QUASIMODO2-LTR_DM
QUASIMODO	QUASIMODO_I	QUASIMODO_LTR
Stalker2	Stalker2_I	Stalker2_LTR
Gypsy3	Gypsy3_I	Gypsy3_LTR
DM1731	DM1731_I	DM173

## Manually inspecting and curing the dictionnary...

In [15]:
TE_dictionnary_file = "example_data/families.dictionnary.curated"


In [9]:
## Generating new fasta with flanked intern parts using dictionnary

def import_dictionnary(TE_dictionnary_file):
    TE_dict = dict()
    with open(TE_dictionnary_file, 'r') as dictionnary:
        for line in dictionnary :
            if len(line.split()) == 3:
                TE_name, I_part, LTR_part = line.split()
                TE_dict[TE_name] = [I_part, LTR_part]
            elif len(line.split()) == 1:
                TE_name = line.strip()
                TE_dict[TE_name] = []
            else :
                raise ValueError("Invalid dictionnary line : \n" + line + "\nEach line should contains either 3 columns corresponding to the TE_name, TE_intern_part, TE_LTR_part OR only contains 1 column corresponding to the TE_name.")
    return TE_dict

In [114]:


def generating_LTR_flanked_fasta_file(consensus_fasta, TE_dictionnary_file):
    TE_dict = import_dictionnary(TE_dictionnary_file)
    I_dict = {v[0] : k for k, v in TE_dict.items() if v}
    LTR_dict = {v[1] : k for k, v in TE_dict.items() if v}

    record_iterator = SeqIO.parse(consensus_fasta, "fasta")
    reconstructed_records = dict()
    new_records = []
    for record in record_iterator:
        TE_ID = get_TE_ID(record.description)
        if TE_ID in TE_dict: # If TE_ID directly in keys, then it means its a non_splitted TE
            record.id = TE_ID
            record.description = ""
            new_records.append(record)
        elif TE_ID in I_dict:
            if I_dict[TE_ID] not in reconstructed_records :
                reconstructed_records[I_dict[TE_ID]] = [record, None]
            else :
                reconstructed_records[I_dict[TE_ID]][0] = record
        elif TE_ID in LTR_dict:
            if LTR_dict[TE_ID] not in reconstructed_records :
                reconstructed_records[LTR_dict[TE_ID]] = [None, record]
            else :
                reconstructed_records[LTR_dict[TE_ID]][1] = record
        else:
            raise ValueError
    for merged_TE_ID, TE_parts_list in reconstructed_records.items() :
        I_part_record, LTR_part_record = TE_parts_list
        new_record = I_part_record
        new_record.seq = LTR_part_record.seq + I_part_record.seq + LTR_part_record.seq

        # merged_TE_ID = get_TE_hierarchy_from_embl_file(merged_TE_ID, embl_hierarchy_dict)

        new_record.id = merged_TE_ID
        new_record.description = ""
        new_records.append(new_record)
    SeqIO.write(new_records, "my_example.faa", "fasta")

generating_LTR_flanked_fasta_file(Dfam_consensus_fasta, TE_dictionnary_file)

## Next cell if just custom code to add hierarchy of TE in TE_name

In [162]:
embl_file = "example_data/families.embl"
TE_dict = import_dictionnary(TE_dictionnary_file)

class EmblRecord:
    def __init__(self, acc, name, hierarchy):
        self.acc = acc
        self.name = name
        self.hierarchy = hierarchy

def embl_parser(embl_file):
    custom_embl_dict = {}
    with open(embl_file,'r') as input:
        for line in input:
            if line.startswith("CC"):
                if line.startswith("CC        Type:"):
                    TE_type = line.split(":")[1]
                if line.startswith("CC        SubType:"):
                    TE_subtype = line.split(":")[1]
                continue
            if line.startswith("AC"):
                acc = line.split()[-1].replace(";", "")
            if line.startswith("NM"):
                name = line.split()[-1]
            if line.startswith("//"):
                if not TE_subtype.strip():
                    TE_subtype = "Unknown"
                hierarchy = TE_type.strip() + "/" + TE_subtype.strip()

                new_record = EmblRecord(acc, name, hierarchy)
                custom_embl_dict[name] = new_record
    return custom_embl_dict

def build_TE_hierarchy_dict_from_embl_file(TE_dict, embl_file):
    embl_dict = embl_parser(embl_file)
    TE_hierarchy_dict = dict()
    for TE, TE_parts in TE_dict.items():
        if TE_parts:
            record = embl_dict[TE_parts[0]]
        elif TE in embl_dict:
            record = embl_dict[TE]
        else:
            logging.warning("TE `" + TE + "` could not be match in EMBL file. No hierarchy will be added.")
            continue

        TE_hierarchy_dict[TE] = TE + '#'+ record.hierarchy
    return TE_hierarchy_dict

TE_hierarchy_dict = build_TE_hierarchy_dict_from_embl_file(TE_dict, embl_file)

def generating_LTR_flanked_fasta_file_with_hierarchy(consensus_fasta, TE_dictionnary_file, TE_hierarchy_dict):
    TE_dict = import_dictionnary(TE_dictionnary_file)
    I_dict = {v[0] : k for k, v in TE_dict.items() if v}
    LTR_dict = {v[1] : k for k, v in TE_dict.items() if v}

    record_iterator = SeqIO.parse(consensus_fasta, "fasta")
    reconstructed_records = dict()
    new_records = []
    for record in record_iterator:
        TE_ID = record.description.split()[-1]
        # print(TE_ID)
        # print(TE_hierarchy_dict[LTR_dict[TE_ID]])
        # break
        if TE_ID in TE_dict: # If TE_ID directly in keys, then it means its a non_splitted TE
            record.id = TE_hierarchy_dict[TE_ID]
            record.description = ""
            new_records.append(record)
        elif TE_ID in I_dict:
            if I_dict[TE_ID] not in reconstructed_records :
                reconstructed_records[I_dict[TE_ID]] = [record, None]
            else :
                reconstructed_records[I_dict[TE_ID]][0] = record
        elif TE_ID in LTR_dict:
            if LTR_dict[TE_ID] not in reconstructed_records :
                reconstructed_records[LTR_dict[TE_ID]] = [None, record]
            else :
                reconstructed_records[LTR_dict[TE_ID]][1] = record
        else:
            raise ValueError
    for merged_TE_ID, TE_parts_list in reconstructed_records.items() :
        I_part_record, LTR_part_record = TE_parts_list
        new_record = I_part_record
        new_record.seq = LTR_part_record.seq + I_part_record.seq + LTR_part_record.seq
        new_record.id = TE_hierarchy_dict[merged_TE_ID]
        new_record.description = ""
        new_records.append(new_record)
    SeqIO.write(new_records, "example_data/families.flanked_LTR.hierarchy.fa", "fasta")

generating_LTR_flanked_fasta_file_with_hierarchy(Dfam_consensus_fasta, TE_dictionnary_file, TE_hierarchy_dict)

### Checking if it went well...

In [110]:
record_iterator = SeqIO.parse(Dfam_consensus_fasta, "fasta")

for record in record_iterator:
    if record.description == "DF0001619.1 Gypsy2_I":
        print(record)
    if record.description == "DF0001620.1 Gypsy2-I_DM":
        print(record)
    if record.description == "DF0001621.1 Gypsy2_LTR":
        print(record)
    if record.description == "DF0001622.1 Gypsy2-LTR_DM":
        print(record)


ID: DF0001619.1
Name: DF0001619.1
Description: DF0001619.1 Gypsy2_I
Number of features: 0
Seq('ggcgcccaaccagtggtatttgacagtgcgttagtcattacccacgaacaaaaa...ggt')
ID: DF0001620.1
Name: DF0001620.1
Description: DF0001620.1 Gypsy2-I_DM
Number of features: 0
Seq('gggaccagcgaataacgcgtacgacagacaaaattctaagtcgcgaagcaaaat...tat')
ID: DF0001621.1
Name: DF0001621.1
Description: DF0001621.1 Gypsy2_LTR
Number of features: 0
Seq('agttaaccaaagctaacgtcgctcccacgggcgtatgaaatacgacaacancgg...att')
ID: DF0001622.1
Name: DF0001622.1
Description: DF0001622.1 Gypsy2-LTR_DM
Number of features: 0
Seq('tgccctacaagtnattgtcccacatcttgtntttcaacatcgtttctgggtaaa...aca')


In [113]:
record_iterator = SeqIO.parse(Dfam_consensus_fasta, "fasta")

for record in record_iterator:
    if record.description == "DF0001525.1 ZAM_LTR":
        print(record.seq)
    if record.description == "DF0001526.2 ZAM_I":
        print(record.seq)

agttaccgacccatcggtaccatacaccacccctccctctaaaccaccacgcctacacaagtagaagacatcgaaccgggaagctttgcgatacaaagtcgcaacataaacatcaacgacgagtcagccgccgacatccgcccaaaaagctgacaccgcattcttttcgctcagacagggcaacgcatacaattccatatacatacttataaacatactcatactttctgctgtgtcagatactttatttctaagaactttaacattgtaacacacacatacatattcactgttagcttatttaagaccaagaataaagacgacgacagtcgagntcgagcagcaagcacttgtggacgtatctaatctccgatcaaaattctctggagacagccgaggctacattctggatcgatacaactcctctctcccgctgtattanaatacctccgcgaagctccccggaggtaac
tggcgcagccgggaaactggaatggaaaatactctattaaaccttttattagttctattgtaagtagttgtggaaaaagagtgagaatgaagtgcagaaatgtctaaaagtgattacaacaaaagtcctactacaatacataaaccgccttaacaaacatacaaaacacanatataaangnaaaaaaaaaaaaaaaaanaaaaannaanacccnaaacttaaaaatgccgtaatcgtgaaacatgatatgtgttgtacttgcgtggnatcaatcgctgataatcactgccgaaatttattaaggccaagtaccacatcantactctcacgtatacatacatatatatgcnnnacaattaaaacaacatacacacacacaaatatttcaaatgcaaaaaaanangaangaatgtagtgtacctgcgtggcatcaatcgctgataaaccactgccgaaatattaaaggcccggtactacatcacaaaacanatatatatgcaacaaaaatatacacaacaaaaccatat